In [ ]:
import os
import fim
import torch
import numpy as np

from torch import nn
from torch.utils.data import DataLoader
from fim.data.dataloaders import (
    DataLoaderFactory
)
from fim.utils.helper import (
    GenericConfig, 
    expand_params, 
    load_yaml
)

from fim.data.datasets import FIMSDEDataset,FIMSDEDatabatchTuple
from fim.data.config_dataclasses import FIMDatasetConfig
from fim.models.blocks import ModelFactory
from fim.models.config_dataclasses import FIMSDEConfig
from fim.data.config_dataclasses import FIMDatasetConfig
from fim.models.sde import FIMSDE
from fim.utils.experiment_files import ExperimentsFiles


In [1]:
experiment_dir = r"C:\Users\cesar\Desktop\Projects\FoundationModels\FIM\results\1732042941"
experiments_files = ExperimentsFiles(experiment_dir=experiment_dir)

NameError: name 'ExperimentsFiles' is not defined